In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
#Load the data.
train = pd.read_json("data/processed/train.json")
test = pd.read_json("data/processed/test.json")

In [3]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        #Create 3 bands having HH, HV and avg of both
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale, be carefull when testing cause it should do the same rescale to test data 
        #using mean, max and min value of train data
        #a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        #b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        #c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        #imgs.append(np.dstack((a, b, c)))
        
        imgs.append(np.dstack((band_1, band_2, band_3)))

    return np.array(imgs)

In [54]:
train[ 'inc_angle' ] = train[ 'inc_angle' ].replace('na', 0)
train[ 'inc_angle' ] = train[ 'inc_angle' ].astype(float).fillna(0.0)

indx_tr = np.where(train.inc_angle > 0)
print (indx_tr[0].shape)

#Generate the training data
X_train_valid = get_scaled_imgs(train)
#X_test = get_scaled_imgs(test)
train_valid_a = train[ 'inc_angle' ]
y_train_valid =train['is_iceberg']

y_train_valid = y_train_valid[indx_tr[0]]
X_train_valid = X_train_valid[indx_tr[0], ...]
train_valid_a = train_valid_a[indx_tr[0]]

print (X_train_valid.shape)
print (y_train_valid.shape)
print(train_valid_a.shape)

(1471,)
(1471, 75, 75, 3)
(1471,)
(1471,)


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, random_state=1, train_size=0.75)

E:\software\anaconda\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import ndarray as nd
from mxnet.gluon.data import vision
import mxnet as mx

In [71]:
from tqdm import tqdm
from skimage.transform import resize

def resize_images(df, width):
    n = len(df)
    df_resized = np.zeros((n, width, width, 3), dtype=np.float32)
    for i in tqdm(range(n)):
        x = df[i]
        x = (x-x.min())/(x.max()-x.min()) # normalize for each pseudo pixel value
        df_resized[i] = resize(x, (width,width), mode='reflect')
    return df_resized

In [118]:
ctx = mx.gpu()

train_augs = [
    #image.ForceResizeAug((224,224)),
    #image.LightingAug(alphastd=0.5),
    image.HorizontalFlipAug(p=0.5),
    image.ColorJitterAug(brightness=0.5, contrast=0.5, saturation=0.5),
    image.ColorNormalizeAug(mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225]))
]

def transform_train(data, label):
    data = data.astype('float32')
    for pre in train_augs:
        data = pre(data)
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    data = nd.transpose(data, (0, 3, 1, 2))
    #label = nd.transpose(nd.array([label]), (1,0))
    #label = label.transpose()
    return (data, nd.array(label.astype('float32')))

In [119]:
def transform_test(data, label):
    data = data.astype('float32')
    
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    data = nd.transpose(data, (0, 3, 1, 2))
    #label = nd.transpose(nd.array([label]), (1,0))
    #label = label.transpose(())
    return (data, nd.array(label.astype('float32')))

In [81]:
X_train = resize_images(X_train, 224)
X_valid = resize_images(X_valid, 224)

100%|███████████████████████████████████████████████████████████████████████████████| 368/368 [00:02<00:00, 127.29it/s]


In [82]:
X_train.shape

(1103, 224, 224, 3)

In [117]:
 nd.array(y_train.astype('float32')).shape

(1103,)

In [120]:
train_da = transform_train(nd.array(X_train), y_train)
valid_da = transform_test(nd.array(X_valid), y_valid)
train_valid_da = transform_train(nd.array(X_train_valid), y_train_valid)

In [101]:
X_train_da[0].shape

(1103, 3, 224, 224)

In [102]:
X_train_da[1].shape

(1103, 1)

In [122]:
batch_size = 64

loader = gluon.data.DataLoader
train_data = loader(train_da, batch_size, shuffle=True, last_batch='keep')
valid_data = loader(valid_da, batch_size, shuffle=True, last_batch='keep')
train_valid_data = loader(train_valid_da, batch_size, shuffle=True,
                          last_batch='keep')

In [58]:
from mxnet.gluon import nn
from mxnet import nd

class Residual(nn.HybridBlock):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        with self.name_scope():
            strides = 1 if same_shape else 2
            self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1,
                                  strides=strides)
            self.bn1 = nn.BatchNorm()
            self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm()
            if not same_shape:
                self.conv3 = nn.Conv2D(channels, kernel_size=1,
                                      strides=strides)

    def hybrid_forward(self, F, x):
        #out = F.relu(self.bn1(self.conv1(x)))
        #out = self.bn2(self.conv2(out))
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(self.bn2(out))
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(out + x)


class ResNet(nn.HybridBlock):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            net = self.net = nn.HybridSequential()
            # 模块1
            net.add(nn.Conv2D(channels=32, kernel_size=3, strides=1,
                              padding=1))
            net.add(nn.BatchNorm())
            net.add(nn.Activation(activation='relu'))
            # 模块2
            for _ in range(3):
                net.add(Residual(channels=32))
            # 模块3
            net.add(Residual(channels=64, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=64))
            # 模块4
            net.add(Residual(channels=128, same_shape=False))
            for _ in range(2):
                net.add(Residual(channels=128))
            # 模块5
            net.add(nn.GlobalAvgPool2D())
            net.add(nn.Flatten())
            net.add(nn.Dense(num_classes))

    def hybrid_forward(self, F, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s'%(i+1, out.shape))
        return out


def get_net(ctx):
    num_outputs = 2
    net = ResNet(num_outputs)
    net.initialize(ctx=ctx, init=init.Xavier())
    return net

In [60]:
import datetime
import sys
sys.path.append('..')
#import utils

def get_loss(data, net, ctx):
    loss = 0.0
    for feas, label in data:
        label = label.as_in_context(ctx)
        output = net(feas.as_in_context(ctx))
        cross_entropy = softmax_cross_entropy(output, label)
        loss += nd.mean(cross_entropy).asscalar()
    return loss / len(data)

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period,
          lr_decay):
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9,
                                      'wd': wd})
    prev_time = datetime.datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0.0
        if epoch > 0 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = get_loss(valid_data, net, ctx)
            epoch_str = ("Epoch %d. Train loss: %f, Valid loss %f, "
                         % (epoch, train_loss / len(train_data), valid_loss))
        else:
            epoch_str = ("Epoch %d. Train loss: %f, "
                         % (epoch, train_loss / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))

In [123]:
ctx = mx.gpu()
num_epochs = 100
learning_rate = 0.001
weight_decay = 5e-4
lr_period = 40
lr_decay = 0.1

net = get_net(ctx)
net.hybridize()
train(net, train_data, valid_data, num_epochs, learning_rate,
      weight_decay, ctx, lr_period, lr_decay)

MXNetError: [16:03:43] d:\program files (x86)\jenkins\workspace\mxnet\mxnet\include\mxnet\./tensor_blob.h:276: Check failed: this->shape_.Size() == shape.Size() (166032384 vs. 1103) TBlob.get_with_shape: new and old shape do not match total elements